In [1]:
# Instalar PySpark en Google Colab
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9b2bc7b60540aced7ffd5d6afc2a60742a580be50a3d76d009ad9079bb550096
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pandas as pd

In [3]:
# Usar pandas para descargar el archivo
url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CPIAUCSL&scale=left&cosd=1947-01-01&coed=2024-07-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-08-31&revision_date=2024-08-31&nd=1947-01-01"

# Descargar el CSV
cpi_data = pd.read_csv(url)

# Guardar el archivo localmente
cpi_data.to_csv("CPIAUCSL.csv", index=False)


Acá empieza la parte de SQL/PySpark

In [4]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("SQL Basics").getOrCreate()

In [5]:
# Cargar el dataset desde el archivo CSV local
cpi_df = spark.read.csv("CPIAUCSL.csv", header=True, inferSchema=True)

In [6]:
# Mostrar la estructura del DataFrame
cpi_df.printSchema()

root
 |-- DATE: date (nullable = true)
 |-- CPIAUCSL: double (nullable = true)



In [7]:
# Mostrar las primeras filas del dataset
cpi_df.show(5)

+----------+--------+
|      DATE|CPIAUCSL|
+----------+--------+
|1947-01-01|   21.48|
|1947-02-01|   21.62|
|1947-03-01|    22.0|
|1947-04-01|    22.0|
|1947-05-01|   21.95|
+----------+--------+
only showing top 5 rows



In [8]:
# Registrar el DataFrame como una tabla SQL temporal
cpi_df.createOrReplaceTempView("cpi_data")

In [18]:
# Consultar información básica del dataset
spark.sql("SELECT COUNT(*) AS guille FROM cpi_data").show()

+------+
|guille|
+------+
|   931|
+------+



In [24]:
# Calcular estadísticas descriptivas (media, máximo, mínimo) para el índice de precios al consumidor (CPIAUCSL)
spark.sql("SELECT AVG(CPIAUCSL) AS federico, MAX(CPIAUCSL) AS max_cpi, MIN(CPIAUCSL) AS min_cpi, MEDIAN(CPIAUCSL) AS median_cpi  FROM cpi_data").show()

+------------------+-------+-------+----------+
|          federico|max_cpi|min_cpi|median_cpi|
+------------------+-------+-------+----------+
|120.47689366272824|313.534|  21.48|     108.5|
+------------------+-------+-------+----------+



In [23]:
# Filtrar datos para observar el CPI después del año 2000
spark.sql("""
    SELECT *
    FROM cpi_data
    WHERE DATE >= '2020-01-01'
""").show()

+----------+--------+
|      DATE|CPIAUCSL|
+----------+--------+
|2020-01-01| 258.906|
|2020-02-01| 259.246|
|2020-03-01|  258.15|
|2020-04-01| 256.126|
|2020-05-01| 255.848|
|2020-06-01| 257.004|
|2020-07-01| 258.408|
|2020-08-01| 259.366|
|2020-09-01| 259.951|
|2020-10-01| 260.249|
|2020-11-01| 260.895|
|2020-12-01| 262.005|
|2021-01-01| 262.518|
|2021-02-01| 263.583|
|2021-03-01|  264.91|
|2021-04-01| 266.752|
|2021-05-01| 268.452|
|2021-06-01| 270.664|
|2021-07-01| 271.994|
|2021-08-01| 272.789|
+----------+--------+
only showing top 20 rows



In [25]:
# Extraer el año y calcular el CPI promedio por año
spark.sql("""
    SELECT
        SUBSTRING(DATE, 1, 4) AS Year,
        AVG(CPIAUCSL) AS average_cpi
    FROM cpi_data
    GROUP BY Year
    ORDER BY Year
""").show()


+----+------------------+
|Year|       average_cpi|
+----+------------------+
|1947|22.331666666666667|
|1948|            24.045|
|1949|23.809166666666666|
|1950|           24.0625|
|1951| 25.97333333333333|
|1952|26.566666666666666|
|1953|26.768333333333334|
|1954|            26.865|
|1955|26.795833333333334|
|1956| 27.19083333333333|
|1957|28.113333333333333|
|1958| 28.88083333333334|
|1959| 29.15000000000001|
|1960|29.585000000000004|
|1961| 29.90166666666667|
|1962|30.253333333333334|
|1963|30.633333333333336|
|1964| 31.03833333333333|
|1965|31.528333333333332|
|1966| 32.47083333333333|
+----+------------------+
only showing top 20 rows



In [26]:
# Agrupar por década y calcular el CPI promedio
spark.sql("""
    SELECT
        CONCAT(SUBSTRING(DATE, 1, 3), '0s') AS Decade,
        AVG(CPIAUCSL) AS average_cpi
    FROM cpi_data
    GROUP BY Decade
    ORDER BY Decade
""").show()

+------+------------------+
|Decade|       average_cpi|
+------+------------------+
| 1940s|23.395277777777775|
| 1950s|27.036666666666658|
| 1960s| 32.02608333333334|
| 1970s| 52.40749999999999|
| 1980s|104.64916666666666|
| 1990s| 149.9191666666667|
| 2000s|193.59964166666666|
| 2010s|        237.113225|
| 2020s| 285.6651636363637|
+------+------------------+



In [ ]:
# Calcular el cambio porcentual mensual en el CPI
spark.sql("""
    SELECT
        DATE,
        CPIAUCSL,
        (CPIAUCSL - LAG(CPIAUCSL, 1) OVER (ORDER BY DATE)) / LAG(CPIAUCSL, 1) OVER (ORDER BY DATE) AS monthly_change
    FROM cpi_data
    ORDER BY DATE
""").show()


+----------+--------+--------------------+
|      DATE|CPIAUCSL|      monthly_change|
+----------+--------+--------------------+
|1947-01-01|   21.48|                NULL|
|1947-02-01|   21.62|0.006517690875232801|
|1947-03-01|    22.0| 0.01757631822386674|
|1947-04-01|    22.0|                 0.0|
|1947-05-01|   21.95|-0.00227272727272...|
|1947-06-01|   22.08|0.005922551252847335|
|1947-07-01|   22.23|0.006793478260869662|
|1947-08-01|    22.4|0.007647323436797038|
|1947-09-01|   22.84|  0.0196428571428572|
|1947-10-01|   22.91|0.003064798598949...|
|1947-11-01|   23.06|0.006547359231776455|
|1947-12-01|   23.41|0.015177797051170921|
|1948-01-01|   23.68| 0.01153353267834257|
|1948-02-01|   23.67|-4.22297297297213...|
|1948-03-01|    23.5|-0.00718208702999...|
|1948-04-01|   23.82|0.013617021276595757|
|1948-05-01|   24.01|0.007976490344248584|
|1948-06-01|   24.15|0.005830903790087...|
|1948-07-01|    24.4|0.010351966873706004|
|1948-08-01|   24.43|0.001229508196721...|
+----------

In [ ]:
# Encontrar las fechas con el CPI más alto y más bajo
spark.sql("""
    SELECT DATE, CPIAUCSL
    FROM cpi_data
    WHERE CPIAUCSL = (SELECT MAX(CPIAUCSL) FROM cpi_data)
       OR CPIAUCSL = (SELECT MIN(CPIAUCSL) FROM cpi_data)
""").show()


+----------+--------+
|      DATE|CPIAUCSL|
+----------+--------+
|1947-01-01|   21.48|
|2024-07-01| 313.534|
+----------+--------+



In [ ]:
# Calcular la media móvil (rolling average) de 12 meses del CPI
spark.sql("""
    SELECT
        DATE,
        CPIAUCSL,
        AVG(CPIAUCSL) OVER (ORDER BY DATE ROWS BETWEEN 11 PRECEDING AND CURRENT ROW) AS moving_avg
    FROM cpi_data
""").show()


+----------+--------+------------------+
|      DATE|CPIAUCSL|        moving_avg|
+----------+--------+------------------+
|1947-01-01|   21.48|             21.48|
|1947-02-01|   21.62|             21.55|
|1947-03-01|    22.0|              21.7|
|1947-04-01|    22.0|            21.775|
|1947-05-01|   21.95|             21.81|
|1947-06-01|   22.08|            21.855|
|1947-07-01|   22.23|21.908571428571427|
|1947-08-01|    22.4|             21.97|
|1947-09-01|   22.84|22.066666666666666|
|1947-10-01|   22.91|            22.151|
|1947-11-01|   23.06|22.233636363636364|
|1947-12-01|   23.41|22.331666666666667|
|1948-01-01|   23.68|            22.515|
|1948-02-01|   23.67|22.685833333333335|
|1948-03-01|    23.5|22.810833333333335|
|1948-04-01|   23.82|22.962500000000002|
|1948-05-01|   24.01| 23.13416666666667|
|1948-06-01|   24.15| 23.30666666666666|
|1948-07-01|    24.4|23.487499999999994|
|1948-08-01|   24.43|23.656666666666666|
+----------+--------+------------------+
only showing top

In [ ]:
# Crear un índice simple basado en un año base (por ejemplo, 2000)
spark.sql("""
    WITH base_year AS (
        SELECT AVG(CPIAUCSL) AS base_cpi
        FROM cpi_data
        WHERE DATE LIKE '2000%'
    )
    SELECT
        DATE,
        CPIAUCSL,
        CPIAUCSL / (SELECT base_cpi FROM base_year) * 100 AS cpi_index
    FROM cpi_data
""").show()


+----------+--------+------------------+
|      DATE|CPIAUCSL|         cpi_index|
+----------+--------+------------------+
|1947-01-01|   21.48|12.474471277162076|
|1947-02-01|   21.62| 12.55577602477859|
|1947-03-01|    22.0|12.776460339737694|
|1947-04-01|    22.0|12.776460339737694|
|1947-05-01|   21.95|12.747422929874652|
|1947-06-01|   22.08|12.822920195518556|
|1947-07-01|   22.23|12.910032425107678|
|1947-08-01|    22.4|13.008759618642015|
|1947-09-01|   22.84|13.264288825436768|
|1947-10-01|   22.91|13.304941199245027|
|1947-11-01|   23.06|13.392053428834144|
|1947-12-01|   23.41|13.595315297875427|
|1948-01-01|   23.68|13.752117311135844|
|1948-02-01|   23.67|13.746309829163236|
|1948-03-01|    23.5|13.647582635628899|
|1948-04-01|   23.82|13.833422058752356|
|1948-05-01|   24.01| 13.94376421623191|
|1948-06-01|   24.15|14.025068963848422|
|1948-07-01|    24.4|14.170256013163623|
|1948-08-01|   24.43|14.187678459081448|
+----------+--------+------------------+
only showing top